In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.09873366355895996
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 19.247269668215722
200000 31.936668461158884
300000 31.093019086352196
400000 31.631703743987615
500000 31.11230016028229
600000 32.83886408516379
700000 31.328457147896266
800000 30.35865679992095
900000 31.41752246089448
1000000 30.22541212597391
1100000 13.57116638743624
1200000 27.882298336235245
1300000 30.35847570632169
1400000 31.25022242902117
1500000 30.308705426762305
1600000 31.14488531300543
1700000 32.0511725136071
1800000 30.196911520341114
1900000 31.20172308062593
2000000 30.844328568711596
2100000 30.623965179379052
2200000 8.341133634305729
2300000 35.196454177741956
2400000 38.90444754170949
2500000 37.13236187960002
2600000 36.87914271895723
2700000 37.05373023683881
2800000 35.26055187638182
2900000 35.67193298937864
3000000 34.7994728

29600000 29.76042958451515
29700000 34.06722326120678
29800000 28.950241028436952
29900000 16.60599319561454
30000000 33.08254733398322
30100000 30.038751307201633
30200000 32.42388782933966
30300000 20.420825353220774
30400000 22.980502264423016
30500000 33.85736709967371
30600000 29.453977394364802
30700000 33.81953714022639
30800000 30.7961701658492
30900000 29.72311890073224
31000000 29.402657956312105
31100000 25.017133421390795
31200000 13.092450886770035
31300000 28.222743385182316
31400000 29.237251045127817
31500000 30.085391473273834
31600000 28.75227473538503
31700000 32.293983342012666
31800000 31.96434805960821
31900000 30.67142647506227
32000000 12.797699123697479
32100000 31.193247582326542
32200000 31.79746957974066
32300000 31.715398288240706
32400000 31.34584356870656
32500000 11.184580052011539
32600000 31.317995471867892
32700000 32.92352771052722
32800000 31.84668960022584
32900000 29.618768018289856
33000000 30.240441697935946
33100000 28.004255002025594
33200000 

59300000 24.499771092535433
59400000 23.731336779162874
59500000 24.515521413354133
59600000 25.883756306110207
59700000 11.406673641044826
59800000 28.09690857772433
59900000 26.487319513176438
60000000 27.120041015048468
60100000 27.37307128219471
60200000 27.84158709167296
60300000 28.41371926762863
60400000 27.459007918136603
60500000 27.678528003771813
60600000 15.60507547135146
60700000 22.13083876137062
60800000 26.051642712174193
60900000 25.34896194484549
61000000 25.256289754240072
61100000 24.275309795645178
61200000 12.981928183681523
61300000 21.69097122738415
61400000 24.62358543919064
61500000 24.758097853396265
61600000 24.16033693918538
61700000 25.707753465192738
61800000 15.455279901128616
61900000 23.95758770000848
62000000 27.74491303693725
62100000 27.206895767367868
62200000 28.29604373552048
62300000 27.59072075209117
62400000 28.786715864719717
62500000 26.9945751989215
62600000 27.89550105540573
62700000 28.02578872819104
62800000 10.197389000266464
62900000 2

89100000 12.358117777563317
89200000 18.25701079493652
89300000 21.97441403049652
89400000 22.507979509624406
89500000 11.484658771415276
89600000 22.0035657743853
89700000 21.759943455472275
89800000 22.938004186303314
89900000 23.5873251334071
90000000 24.246228069071165
90100000 24.43309687143857
90200000 24.594089233025198
90300000 24.81319352757961
90400000 26.62630272437286
90500000 24.94634074273798
90600000 25.42813186121001
90700000 23.05683235318894
90800000 22.53625570333865
90900000 10.803837789234098
91000000 20.5887891382392
91100000 24.148089884238285
91200000 21.778523432613998
91300000 8.840138112707296
91400000 23.693344547854025
91500000 22.130675434212577
91600000 17.632273814142984
91700000 17.36095382453058
91800000 22.412002279835516
91900000 23.913979265832157
92000000 22.762993069967823
92100000 25.832744005824168
92200000 24.715833205316603
92300000 24.70740040912446
92400000 25.928130775829928
92500000 23.94362577925827
92600000 25.849494859903004
92700000 24

118200000 24.46472009360453
118300000 22.089738450182303
118400000 22.089183884259533
118500000 21.106478379704352
118600000 21.06743624411569
118700000 21.783939278169644
118800000 20.081470773211446
118900000 21.968872428482488
119000000 12.536935453142021
119100000 16.926339854557693
119200000 7.040424396385889
119300000 13.723361219107227
119400000 16.91981941717418
119500000 21.418947711023023
119600000 20.420183624926267
119700000 21.894049875549324
119800000 20.937165536078144
119900000 21.884820265483338
120000000 21.212330546614776
120100000 21.630481510861184
120200000 25.12592359275678
120300000 22.218987796805138
120400000 24.288301184238204
120500000 21.775826658649187
120600000 20.999291276499918
120700000 21.82023931146617
120800000 20.653497408290434
120900000 21.44923257729629
121000000 21.75846103224285
121100000 19.577418191196386
121200000 7.963167304034518
121300000 12.606279390573357
121400000 14.601438644643139
121500000 13.566045829273076
121600000 19.7689507016

...

290200000 4.463731356353046
290300000 5.11454520935546
290400000 4.961569396834446
290500000 5.060469496329879
290600000 5.014921115634674
290700000 4.68589124963942
290800000 3.5479701617969557
290900000 1.0960443515753766
291000000 1.098554049956755
291100000 5.645262606007485
291200000 4.551937911659537
291300000 5.064546537071386
291400000 5.028120107133469
291500000 4.706964542352557
291600000 4.70022970723302
291700000 4.687732650318627
291800000 5.043080520031045
291900000 4.8684954640185865
292000000 5.869812810143797
292100000 5.21311389184443
292200000 5.022619519699498
292300000 5.110647032766074
292400000 4.5346771273653745
292500000 5.005523483949962
292600000 4.772331352423079
292700000 5.010445036490019
292800000 4.628688162953473
292900000 5.041030024336294
293000000 1.0739286132134658
293100000 1.0712715389602006
293200000 3.8153383290947125
293300000 4.578712627861154
293400000 4.782948680335609
293500000 4.9625470435030605
293600000 4.771745330057731
293700000 4.5462

319300000 4.036893590978835
319400000 4.118227205377222
319500000 3.8791224149659076
319600000 3.923641877590284
319700000 3.7068592016594724
319800000 3.9672982384912703
319900000 5.288200489077928
320000000 5.098038035435419
320100000 4.282289109570087
320200000 4.013314986213403
320300000 3.768998602627337
320400000 4.419366123648652
320500000 4.066831569085433
320600000 4.045710179963986
320700000 3.7553210156824526
320800000 4.8559888014144885
320900000 1.1873293418898145
321000000 0.784242191220508
321100000 2.529319806661687
321200000 4.434749847108292
321300000 3.9129426496641937
321400000 4.387150793106106
321500000 4.309267091313725
321600000 4.3499574114543105
321700000 4.100510861705739
321800000 4.179491486636524
321900000 3.871399041548775
322000000 4.699736537513159
322100000 4.939698826583648
322200000 4.5898348086947145
322300000 4.14775817384054
322400000 4.500694699109767
322500000 4.758082628721041
322600000 4.169646205613912
322700000 4.379143263554149
322800000 4.

348000000 1.3188401995050025
348100000 1.3236937029862073
348200000 1.6513579580576419
348300000 1.7815824021625997
348400000 1.2311616309179327
348500000 1.2906236081302842
348600000 1.2682696268434994
348700000 2.2446038020014925
348800000 0.5002425347554207
348900000 0.3252391989515715
349000000 0.7742454242542537
349100000 1.3473936473359822
349200000 1.3014930056700713
349300000 1.4256038282003516
349400000 2.5172248786224842
349500000 2.2122111455520543
349600000 1.4951064465169266
349700000 1.3668570157523074
349800000 1.3613151729540334
349900000 1.441118590748569
350000000 1.914840867825043
350100000 1.3779376921937208
350200000 1.2784409803998047
350300000 1.232988910221966
350400000 1.4067067827332753
350500000 1.2392781504877064
350600000 1.2710412739617327
350700000 1.2698531966398225
350800000 1.2372298873357825
350900000 1.0473827714462698
351000000 0.32161305001902113
351100000 0.3163458293903298
351200000 1.6077594988945085
351300000 1.1961355992884046
351400000 1.2367

376000000 0.16363845161677998
376100000 0.163240889547354
376200000 0.16805096869149735
376300000 0.16248903607934118
376400000 0.16217981762402456
376500000 0.16168496364446414
376600000 0.164694560926389
376700000 0.16344384252518615
376800000 0.1627729333305571
376900000 0.16273698886865376
377000000 0.1669703793536246
377100000 0.16183823457449212
377200000 0.16388335637784468
377300000 0.16843090216434797
377400000 0.1630554231266088
377500000 0.1626350660476638
377600000 0.16197931766751805
377700000 0.1611320230327143
377800000 0.16481707738941048
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 0.9392165863444745
378000000 0.9636603420667251
378100000 1.1097710996462313
378200000 1.0745541855171878
378300000 0.9514779557899515
378400000 0.9008041887535373
378500000 0.927445178664132
378600000 1.555685259285762
378700000 1.0755177604137447
378800000 0.8610483376468003
378900000 1.026880880867801
379000000 1.157621393240024
379100000 0.861982665200

404100000 0.6565521344218486
404200000 0.7669565346303782
404300000 0.7300653996916632
404400000 0.7715518064462714
404500000 0.7551132965616312
404600000 0.6525334021604836
404700000 0.8296220128209684
404800000 0.6618667678976389
404900000 0.7136809027838528
405000000 0.7732309017602258
405100000 0.7653456020308885
405200000 0.7286162918967883
405300000 0.6340281344485502
405400000 0.7922659487459739
405500000 0.6846270168912032
405600000 0.6710924301498467
405700000 0.7635816819803967
405800000 0.7267558405619502
405900000 0.7103576752195087
406000000 0.6181350891613047
406100000 0.7352380155565639
406200000 0.6962803058901953
406300000 0.6478681927089877
406400000 0.6966618211325685
406500000 0.7030085217908772
406600000 0.6300977539594763
406700000 0.5883001431554497
406800000 0.646834518180865
406900000 0.679502668191439
407000000 0.6112248197043558
407100000 0.6071574181166881
407200000 0.7776337065275536
407300000 0.667869535398799
407400000 0.621688940221452
407500000 0.651888